In [5]:
#MODIFIED LOSS
#MIXUP
#image with padding 1

%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os 
os.chdir('..')
import fastai
from fastai.vision import *
from fastai.callbacks import SaveModelCallback
from fastai.callbacks import TrackerCallback
from fastai.callbacks import CSVLogger
from fastai.data_block import _maybe_squeeze
from fastai.callbacks import *
from utils.mxresnet import *
from utils.ranger import *
from sklearn.model_selection import StratifiedKFold, KFold
from joblib import load, dump
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import pretrainedmodels


import albumentations
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations import functional as F
import cv2

In [6]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [7]:
FOLD = 1
NAME = 'EXP_500'
SUFFIX =f'RESNEX_1CH_MISH_SIMPLE_CUTMIX_CUTOUT_MIXUP_with_CROP_NO_CROP_CV6_FOLD_LOSS_GRIDMASK{FOLD}'
PATH = Path('../../../bengaliai')
SZ = 128
BS = 256
NFOLDS = 5 #keep the same split as the initial dataset
SEED = 2019
TRAIN_IMG = PATH/'img_trn_orig'
LABELS = PATH/'train.csv'
TRAIN_IMG_CROP = PATH/'img_trn_crop_image'
HEIGHT = 137
WIDTH = 236

os.chdir(NAME)

In [8]:
def multi_strt_split(x, y, n_folds=6, random_seed = 42069, path=Path('')):  
    try: 
        val_name = load('val_idx.joblib')
    except:
        skf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
        val_name = [(val_idx, trn_idx) for trn_idx, val_idx in skf.split(x, y)]
        dump(val_name,'val_idx.joblib')
    return val_name

In [9]:
df = pd.read_csv(str(LABELS))
nunique = list(df.nunique())[1:-1]

In [10]:
val_idx = multi_strt_split(df['image_id'], df[df.columns[1:-1]])[FOLD]

In [11]:
class GridMask(DualTransform):
    """GridMask augmentation for image classification and object detection.
    
    Author: Qishen Ha
    Email: haqishen@gmail.com
    2020/01/29

    Args:
        num_grid (int): number of grid in a row or column.
        fill_value (int, float, lisf of int, list of float): value for dropped pixels.
        rotate ((int, int) or int): range from which a random angle is picked. If rotate is a single int
            an angle is picked from (-rotate, rotate). Default: (-90, 90)
        mode (int):
            0 - cropout a quarter of the square of each grid (left top)
            1 - reserve a quarter of the square of each grid (left top)
            2 - cropout 2 quarter of the square of each grid (left top & right bottom)

    Targets:
        image, mask

    Image types:
        uint8, float32

    Reference:
    |  https://arxiv.org/abs/2001.04086
    |  https://github.com/akuxcw/GridMask
    """

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []
        self.counter = 0

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = F.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        self.counter = self.counter 
        return image

    def get_params_dependent_on_targets(self, params):
        img = params['image']
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

In [ ]:
def read_data(files):
    tmp = []
    for f in files:
        data = pd.read_parquet(f)
        tmp.append(data)
    tmp = pd.concat(tmp)

    data = tmp.iloc[:, 1:].values
    return data

data_train = read_data([str(PATH/'train_image_data_0.parquet'), 
                        str(PATH/'train_image_data_1.parquet'), 
                        str(PATH/'train_image_data_2.parquet'), 
                        str(PATH/'train_image_data_3.parquet')])

In [ ]:
def get_fn(fn):
    fn = str(fn)
    return int(re.findall(r'\d+', fn)[0])

In [ ]:
transforms_train = albumentations.Compose([
    albumentations.OneOf([
        GridMask(num_grid=3, mode=0, rotate = 5),
        GridMask(num_grid=3, mode=1, rotate = 5),
        GridMask(num_grid=3, mode=2, rotate = 5),
    ], p=1.0)
])


In [1]:
class Counter():
    def __init__ (self, total_epoch = 100, val_lenth = 167367):
        self.counter, self.val_lenth= 0, val_lenth
        self.p = 0
        
    def __call__(self):
        self.counter = self.counter + 1
        if self.counter%self.val_lenth == 0:
            self.p = self.p + 0.01
        return min(self.p, 0.8)
        
  
    
    
    def reset(self):
        self.counter = 0

cn = Counter()
def read_data(fn, data = data_train, transform = transforms_train, train_set = val_idx[1]):
    fn = get_fn(fn)
    img = data[fn].reshape(HEIGHT, WIDTH)
    img = cv2.resize(img, (128, 128))
    #p = cn()
    #transform.p = p
    
    if fn in train_set:
        img = transform(image=img)['image']
        
    img = 255- img.astype(np.uint8)  
    img = PIL.Image.fromarray(img).convert('RGB')
    x = pil2tensor(img,np.float32)
    x.div_(255)
    return Image(x)
    
    
    
class BenGli(ImageList):
    def open(self, fn):
        return read_data(fn)

NameError: name 'data_train' is not defined

In [ ]:
#imagenet_stats
data = (BenGli.from_df(df, path=TRAIN_IMG, folder='', suffix='.png', 
        cols='image_id', convert_mode='L')
        .split_by_idxs(valid_idx=val_idx[0], train_idx=val_idx[1])
        .label_from_df(cols=['grapheme_root','vowel_diacritic','consonant_diacritic'])
        .transform(tfms=None, size=(SZ, SZ), resize_method=ResizeMethod.SQUISH, padding_mode='zeros')
        .databunch(bs=BS)).normalize(imagenet_stats)

In [ ]:
data.show_batch()

In [ ]:
class Head(nn.Module):
    def __init__(self, nc, n, ps=0.2):
        super().__init__()
        layers = [AdaptiveConcatPool2d(), Mish(), Flatten()] + \
            bn_drop_lin(nc*2, 512, True, ps, nn.ReLU(inplace=True)) + \
            bn_drop_lin(512, n, True, ps)
        self.fc = nn.Sequential(*layers)
        self._init_weight()
        
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1.0)
                m.bias.data.zero_()
        
    def forward(self, x):
        return self.fc(x)
    
    
    
class CustomHead(nn.Module):
    def __init__(self, arch, cut, nc, n=nunique, pre=False, ps=0.5, activ_mish=True):
        super().__init__()
        self.body=nn.Sequential(*list(arch.children())[:cut])
        self.head1 = Head(nc,n[0])
        self.head2 = Head(nc,n[1])
        self.head3 = Head(nc,n[2])
        if activ_mish: 
            to_Mish(self.head1), to_Mish(self.head2), to_Mish(self.head3)
        
    def forward(self, x):
        x = self.body(x)
        x1 = self.head1(x)
        x2 = self.head2(x)
        x3 = self.head3(x)
        return x1, x2, x3

In [ ]:
arch = pretrainedmodels.se_resnext50_32x4d(num_classes=1000)
model = CustomHead(arch=arch, cut=-2, nc=2048)

In [ ]:
class Loss_combine(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, input, target,reduction='mean'):
        x1,x2,x3 = input
        x1,x2,x3 = x1.float(),x2.float(),x3.float()
        y = target.long()
        return 0.7*F.cross_entropy(x1,y[:,0],reduction=reduction) + 0.1*F.cross_entropy(x2,y[:,1],reduction=reduction) + \
          0.2*F.cross_entropy(x3,y[:,2],reduction=reduction)

In [ ]:
class Metric_idx(Callback):
    def __init__(self, idx, average='macro'):
        super().__init__()
        self.idx = idx
        self.n_classes = 0
        self.average = average
        self.cm = None
        self.eps = 1e-9
        
    def on_epoch_begin(self, **kwargs):
        self.tp = 0
        self.fp = 0
        self.cm = None
    
    def on_batch_end(self, last_output:Tensor, last_target:Tensor, **kwargs):
        last_output = last_output[self.idx]
        last_target = last_target[:,self.idx]
        preds = last_output.argmax(-1).view(-1).cpu()
        targs = last_target.long().cpu()
        
        if self.n_classes == 0:
            self.n_classes = last_output.shape[-1]
            self.x = torch.arange(0, self.n_classes)
        cm = ((preds==self.x[:, None]) & (targs==self.x[:, None, None])) \
          .sum(dim=2, dtype=torch.float32)
        if self.cm is None: self.cm =  cm
        else:               self.cm += cm

    def _weights(self, avg:str):
        if self.n_classes != 2 and avg == "binary":
            avg = self.average = "macro"
            warn("average=`binary` was selected for a non binary case. \
                 Value for average has now been set to `macro` instead.")
        if avg == "binary":
            if self.pos_label not in (0, 1):
                self.pos_label = 1
                warn("Invalid value for pos_label. It has now been set to 1.")
            if self.pos_label == 1: return Tensor([0,1])
            else: return Tensor([1,0])
        elif avg == "micro": return self.cm.sum(dim=0) / self.cm.sum()
        elif avg == "macro": return torch.ones((self.n_classes,)) / self.n_classes
        elif avg == "weighted": return self.cm.sum(dim=1) / self.cm.sum()
        
    def _recall(self):
        rec = torch.diag(self.cm) / (self.cm.sum(dim=1) + self.eps)
        if self.average is None: return rec
        else:
            if self.average == "micro": weights = self._weights(avg="weighted")
            else: weights = self._weights(avg=self.average)
            return (rec * weights).sum()
    
    def on_epoch_end(self, last_metrics, **kwargs): 
        return add_metrics(last_metrics, self._recall())
    
Metric_grapheme = partial(Metric_idx,0)
Metric_vowel = partial(Metric_idx,1)
Metric_consonant = partial(Metric_idx,2)

class Metric_tot(Callback):
    def __init__(self):
        super().__init__()
        self.grapheme = Metric_idx(0)
        self.vowel = Metric_idx(1)
        self.consonant = Metric_idx(2)
        
    def on_epoch_begin(self, **kwargs):
        self.grapheme.on_epoch_begin(**kwargs)
        self.vowel.on_epoch_begin(**kwargs)
        self.consonant.on_epoch_begin(**kwargs)
    
    def on_batch_end(self, last_output:Tensor, last_target:Tensor, **kwargs):
        self.grapheme.on_batch_end(last_output, last_target, **kwargs)
        self.vowel.on_batch_end(last_output, last_target, **kwargs)
        self.consonant.on_batch_end(last_output, last_target, **kwargs)
        
    def on_epoch_end(self, last_metrics, **kwargs): 
        return add_metrics(last_metrics, 0.5*self.grapheme._recall() +
                0.25*self.vowel._recall() + 0.25*self.consonant._recall())

In [2]:
#def cutmix(learn:Learner, alpha:float=1.0, stack_x:bool=False, stack_y:bool=True) -> Learner:
#    "Add cutmix https://arxiv.org/pdf/1905.04899.pdf to `learn`."
#    learn.callback_fns.append(partial(CutMixCallback, alpha=alpha, stack_y=stack_y))
#    return learn
#
#setattr(cutmix, 'cb_fn', CutMixCallback)
#Learner.cutmix = cutmix

learn = Learner(data, model, loss_func=Loss_combine(), opt_func = Over9000, 
        metrics=[Metric_grapheme(),Metric_vowel(),Metric_consonant(),Metric_tot()])

NameError: name 'Learner' is not defined

In [2]:
lr = 1e-1/4

In [3]:
x, y = next(iter(data.train_dl))

NameError: name 'data' is not defined

In [4]:
x.shape

NameError: name 'x' is not defined

In [ ]:
y

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(100, lr, wd=1e-2,  pct_start=0.0,  div_factor=100)

In [3]:
!sudo shutdown

Shutdown scheduled for Tue 2020-02-11 15:40:39 UTC, use 'shutdown -c' to cancel.
